In [1]:
import os
import numpy as np

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

from test_helper import Test
import os.path

fileName = os.path.join('D:/', 'DM_DOCENTE.CSV')

numPartitions = 100
DocenteRawData = sc.textFile(fileName, numPartitions, 8)

In [2]:
# pega amostra e mostra tela
numPoints = DocenteRawData.count()
print numPoints
samplePoints = DocenteRawData.take(1)
print samplePoints

# começa limpeza dos dados
from pyspark.mllib.regression import LabeledPoint
import numpy as np


header = DocenteRawData.take(1)[0] # o cabeçalho é a primeira linha do arquivo
print header

397612
[u'CO_IES|NO_IES|CO_CATEGORIA_ADMINISTRATIVA|DS_CATEGORIA_ADMINISTRATIVA|CO_ORGANIZACAO_ACADEMICA|DS_ORGANIZACAO_ACADEMICA|IN_CAPITAL_IES|CO_DOCENTE_IES|CO_DOCENTE|CO_SITUACAO_DOCENTE|DS_SITUACAO_DOCENTE|CO_ESCOLARIDADE_DOCENTE|DS_ESCOLARIDADE_DOCENTE|CO_REGIME_TRABALHO|DS_REGIME_TRABALHO|IN_SEXO_DOCENTE|DS_SEXO_DOCENTE|NU_ANO_DOCENTE_NASC|NU_MES_DOCENTE_NASC|NU_DIA_DOCENTE_NASC|NU_IDADE_DOCENTE|CO_COR_RACA_DOCENTE|DS_COR_RACA_DOCENTE|CO_PAIS_DOCENTE|CO_NACIONALIDADE_DOCENTE|DS_NACIONALIDADE_DOCENTE|CO_UF_NASCIMENTO|CO_MUNICIPIO_NASCIMENTO|IN_DOCENTE_DEFICIENCIA|IN_DEF_CEGUEIRA|IN_DEF_BAIXA_VISAO|IN_DEF_SURDEZ|IN_DEF_AUDITIVA|IN_DEF_FISICA|IN_DEF_SURDOCEGUEIRA|IN_DEF_MULTIPLA|IN_DEF_INTELECTUAL|IN_ATU_EAD|IN_ATU_EXTENSAO|IN_ATU_GESTAO|IN_ATU_GRAD_PRESENCIAL|IN_ATU_POS_EAD|IN_ATU_POS_PRESENCIAL|IN_ATU_SEQUENCIAL|IN_ATU_PESQUISA|IN_BOLSA_PESQUISA|IN_SUBSTITUTO|IN_EXERCICIO_DT_REF|IN_VISITANTE|IN_VISITANTE_IFES_VINCULO']
CO_IES|NO_IES|CO_CATEGORIA_ADMINISTRATIVA|DS_CATEGORIA_ADMINI

In [3]:
import re
import datetime
from collections import namedtuple

headeritems = header.split('|') # transformar o cabeçalho em lista
print headeritems

[u'CO_IES', u'NO_IES', u'CO_CATEGORIA_ADMINISTRATIVA', u'DS_CATEGORIA_ADMINISTRATIVA', u'CO_ORGANIZACAO_ACADEMICA', u'DS_ORGANIZACAO_ACADEMICA', u'IN_CAPITAL_IES', u'CO_DOCENTE_IES', u'CO_DOCENTE', u'CO_SITUACAO_DOCENTE', u'DS_SITUACAO_DOCENTE', u'CO_ESCOLARIDADE_DOCENTE', u'DS_ESCOLARIDADE_DOCENTE', u'CO_REGIME_TRABALHO', u'DS_REGIME_TRABALHO', u'IN_SEXO_DOCENTE', u'DS_SEXO_DOCENTE', u'NU_ANO_DOCENTE_NASC', u'NU_MES_DOCENTE_NASC', u'NU_DIA_DOCENTE_NASC', u'NU_IDADE_DOCENTE', u'CO_COR_RACA_DOCENTE', u'DS_COR_RACA_DOCENTE', u'CO_PAIS_DOCENTE', u'CO_NACIONALIDADE_DOCENTE', u'DS_NACIONALIDADE_DOCENTE', u'CO_UF_NASCIMENTO', u'CO_MUNICIPIO_NASCIMENTO', u'IN_DOCENTE_DEFICIENCIA', u'IN_DEF_CEGUEIRA', u'IN_DEF_BAIXA_VISAO', u'IN_DEF_SURDEZ', u'IN_DEF_AUDITIVA', u'IN_DEF_FISICA', u'IN_DEF_SURDOCEGUEIRA', u'IN_DEF_MULTIPLA', u'IN_DEF_INTELECTUAL', u'IN_ATU_EAD', u'IN_ATU_EXTENSAO', u'IN_ATU_GESTAO', u'IN_ATU_GRAD_PRESENCIAL', u'IN_ATU_POS_EAD', u'IN_ATU_POS_PRESENCIAL', u'IN_ATU_SEQUENCIAL', u'I

In [4]:
Docente = namedtuple('Docente',headeritems) # gera a namedtuple Docente com os campos de header
print Docente._fields

('CO_IES', 'NO_IES', 'CO_CATEGORIA_ADMINISTRATIVA', 'DS_CATEGORIA_ADMINISTRATIVA', 'CO_ORGANIZACAO_ACADEMICA', 'DS_ORGANIZACAO_ACADEMICA', 'IN_CAPITAL_IES', 'CO_DOCENTE_IES', 'CO_DOCENTE', 'CO_SITUACAO_DOCENTE', 'DS_SITUACAO_DOCENTE', 'CO_ESCOLARIDADE_DOCENTE', 'DS_ESCOLARIDADE_DOCENTE', 'CO_REGIME_TRABALHO', 'DS_REGIME_TRABALHO', 'IN_SEXO_DOCENTE', 'DS_SEXO_DOCENTE', 'NU_ANO_DOCENTE_NASC', 'NU_MES_DOCENTE_NASC', 'NU_DIA_DOCENTE_NASC', 'NU_IDADE_DOCENTE', 'CO_COR_RACA_DOCENTE', 'DS_COR_RACA_DOCENTE', 'CO_PAIS_DOCENTE', 'CO_NACIONALIDADE_DOCENTE', 'DS_NACIONALIDADE_DOCENTE', 'CO_UF_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'IN_DOCENTE_DEFICIENCIA', 'IN_DEF_CEGUEIRA', 'IN_DEF_BAIXA_VISAO', 'IN_DEF_SURDEZ', 'IN_DEF_AUDITIVA', 'IN_DEF_FISICA', 'IN_DEF_SURDOCEGUEIRA', 'IN_DEF_MULTIPLA', 'IN_DEF_INTELECTUAL', 'IN_ATU_EAD', 'IN_ATU_EXTENSAO', 'IN_ATU_GESTAO', 'IN_ATU_GRAD_PRESENCIAL', 'IN_ATU_POS_EAD', 'IN_ATU_POS_PRESENCIAL', 'IN_ATU_SEQUENCIAL', 'IN_ATU_PESQUISA', 'IN_BOLSA_PESQUISA', 'IN_SUB

In [5]:
def ParseDocente(rec):
    # utilizando re.split() vamos capturar nossos valores
    CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, IN_CAPITAL_IES, CO_DOCENTE_IES, CO_DOCENTE, CO_SITUACAO_DOCENTE, DS_SITUACAO_DOCENTE, CO_ESCOLARIDADE_DOCENTE, DS_ESCOLARIDADE_DOCENTE, CO_REGIME_TRABALHO, DS_REGIME_TRABALHO, IN_SEXO_DOCENTE, DS_SEXO_DOCENTE, NU_ANO_DOCENTE_NASC, NU_MES_DOCENTE_NASC, NU_DIA_DOCENTE_NASC, NU_IDADE_DOCENTE, CO_COR_RACA_DOCENTE, DS_COR_RACA_DOCENTE, CO_PAIS_DOCENTE, CO_NACIONALIDADE_DOCENTE, DS_NACIONALIDADE_DOCENTE, CO_UF_NASCIMENTO, CO_MUNICIPIO_NASCIMENTO, IN_DOCENTE_DEFICIENCIA, IN_DEF_CEGUEIRA, IN_DEF_BAIXA_VISAO, IN_DEF_SURDEZ, IN_DEF_AUDITIVA, IN_DEF_FISICA, IN_DEF_SURDOCEGUEIRA, IN_DEF_MULTIPLA, IN_DEF_INTELECTUAL, IN_ATU_EAD, IN_ATU_EXTENSAO, IN_ATU_GESTAO, IN_ATU_GRAD_PRESENCIAL, IN_ATU_POS_EAD, IN_ATU_POS_PRESENCIAL, IN_ATU_SEQUENCIAL, IN_ATU_PESQUISA, IN_BOLSA_PESQUISA, IN_SUBSTITUTO, IN_EXERCICIO_DT_REF, IN_VISITANTE, IN_VISITANTE_IFES_VINCULO = tuple(rec.split('|'))
    

    return Docente(CO_IES, NO_IES, CO_CATEGORIA_ADMINISTRATIVA, DS_CATEGORIA_ADMINISTRATIVA, CO_ORGANIZACAO_ACADEMICA, DS_ORGANIZACAO_ACADEMICA, IN_CAPITAL_IES, CO_DOCENTE_IES, CO_DOCENTE, CO_SITUACAO_DOCENTE, DS_SITUACAO_DOCENTE, CO_ESCOLARIDADE_DOCENTE, DS_ESCOLARIDADE_DOCENTE, CO_REGIME_TRABALHO, DS_REGIME_TRABALHO, IN_SEXO_DOCENTE, DS_SEXO_DOCENTE, NU_ANO_DOCENTE_NASC, NU_MES_DOCENTE_NASC, NU_DIA_DOCENTE_NASC, NU_IDADE_DOCENTE, CO_COR_RACA_DOCENTE, DS_COR_RACA_DOCENTE, CO_PAIS_DOCENTE, CO_NACIONALIDADE_DOCENTE, DS_NACIONALIDADE_DOCENTE, CO_UF_NASCIMENTO, CO_MUNICIPIO_NASCIMENTO, IN_DOCENTE_DEFICIENCIA, IN_DEF_CEGUEIRA, IN_DEF_BAIXA_VISAO, IN_DEF_SURDEZ, IN_DEF_AUDITIVA, IN_DEF_FISICA, IN_DEF_SURDOCEGUEIRA, IN_DEF_MULTIPLA, IN_DEF_INTELECTUAL, IN_ATU_EAD, IN_ATU_EXTENSAO, IN_ATU_GESTAO, IN_ATU_GRAD_PRESENCIAL, IN_ATU_POS_EAD, IN_ATU_POS_PRESENCIAL, IN_ATU_SEQUENCIAL, IN_ATU_PESQUISA, IN_BOLSA_PESQUISA, IN_SUBSTITUTO, IN_EXERCICIO_DT_REF, IN_VISITANTE, IN_VISITANTE_IFES_VINCULO)


In [6]:
# Aplique a função ParseCurso para cada objeto da base
DocenteHeadlessRDD = (DocenteRawData
                    .filter(lambda x: x != header)
                    .map(ParseDocente)
                    )


firstClean = DocenteHeadlessRDD.take(1)[0]
totalRecs = DocenteHeadlessRDD.count()
print firstClean
print totalRecs

Docente(CO_IES=u'1', NO_IES=u'UNIVERSIDADE FEDERAL DE MATO GROSSO', CO_CATEGORIA_ADMINISTRATIVA=u'1', DS_CATEGORIA_ADMINISTRATIVA=u'P\ufffdblica Federal', CO_ORGANIZACAO_ACADEMICA=u'1', DS_ORGANIZACAO_ACADEMICA=u'Universidade', IN_CAPITAL_IES=u'1', CO_DOCENTE_IES=u'559840', CO_DOCENTE=u'669', CO_SITUACAO_DOCENTE=u'1', DS_SITUACAO_DOCENTE=u'Em exerc\ufffdcio', CO_ESCOLARIDADE_DOCENTE=u'5', DS_ESCOLARIDADE_DOCENTE=u'Doutorado', CO_REGIME_TRABALHO=u'2', DS_REGIME_TRABALHO=u'Tempo Integral sem dedica\ufffd\ufffdo exclusiva', IN_SEXO_DOCENTE=u'0', DS_SEXO_DOCENTE=u'Masculino', NU_ANO_DOCENTE_NASC=u'1957', NU_MES_DOCENTE_NASC=u'1', NU_DIA_DOCENTE_NASC=u'28', NU_IDADE_DOCENTE=u'59', CO_COR_RACA_DOCENTE=u'3', DS_COR_RACA_DOCENTE=u'Parda', CO_PAIS_DOCENTE=u'76', CO_NACIONALIDADE_DOCENTE=u'1', DS_NACIONALIDADE_DOCENTE=u'Brasileira', CO_UF_NASCIMENTO=u'51', CO_MUNICIPIO_NASCIMENTO=u'5103007', IN_DOCENTE_DEFICIENCIA=u'0', IN_DEF_CEGUEIRA=u'', IN_DEF_BAIXA_VISAO=u'', IN_DEF_SURDEZ=u'', IN_DEF_AUDIT

In [7]:
# IES com mais docentes

IESDocenteRDD = (DocenteHeadlessRDD
               .map(lambda rec: (rec.NO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )
IESDocenteCount = sorted(IESDocenteRDD.collect(), key=lambda x: -x[1])
print '\nAs 20 IES com mais docentes:\n{0}'.format(IESDocenteCount[:20])


As 20 IES com mais docentes:
[(u'UNIVERSIDADE DE S\ufffdO PAULO', 6286), (u'UNIVERSIDADE PAULISTA', 6183), (u'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO', 4102), (u'UNIVERSIDADE FEDERAL FLUMINENSE', 3701), (u'UNIVERSIDADE FEDERAL DE MINAS GERAIS', 3473), (u'UNIVERSIDADE ESTADUAL PAULISTA J\ufffdLIO DE MESQUITA FILHO', 3457), (u'UNIVERSIDADE FEDERAL DE PERNAMBUCO', 3081), (u'UNIVERSIDADE EST\ufffdCIO DE S\ufffd', 3044), (u'UNIVERSIDADE DE BRAS\ufffdLIA', 3034), (u'UNIVERSIDADE FEDERAL DA BAHIA', 2954), (u'UNIVERSIDADE FEDERAL DE GOI\ufffdS', 2942), (u'UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE', 2927), (u'UNIVERSIDADE TECNOL\ufffdGICA FEDERAL DO PARAN\ufffd', 2908), (u'UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL', 2846), (u'UNIVERSIDADE DO ESTADO DO RIO DE JANEIRO', 2796), (u'UNIVERSIDADE FEDERAL DA PARA\ufffdBA', 2760), (u'UNIVERSIDADE FEDERAL DO PAR\ufffd', 2611), (u'UNIVERSIDADE FEDERAL DE SANTA CATARINA', 2606), (u'PONTIF\ufffdCIA UNIVERSIDADE CAT\ufffdLICA DE MINAS GERAIS', 2467), (u'U

In [8]:
#mostra a quantidade de docentes doutores por IES


IESDocenteRDD = (DocenteHeadlessRDD
               .map(lambda rec: (rec.CO_IES,1))
               .reduceByKey(lambda x,y: x+y)
               )

print IESDocenteRDD.collect()

[(u'11', 473), (u'1788', 70), (u'1542', 259), (u'803', 43), (u'3962', 182), (u'15704', 33), (u'13467', 16), (u'11007', 20), (u'1771', 6), (u'4026', 107), (u'825', 46), (u'2200', 83), (u'3522', 27), (u'15696', 56), (u'810', 68), (u'68', 45), (u'1717', 108), (u'3975', 7), (u'3643', 2), (u'2275', 59), (u'16543', 9), (u'976', 32), (u'3974', 482), (u'213', 139), (u'15697', 28), (u'10', 1649), (u'12869', 38), (u'18023', 13), (u'14724', 459), (u'3039', 13), (u'1716', 100), (u'1789', 43), (u'3523', 39), (u'4030', 117), (u'802', 168), (u'824', 106), (u'4355', 10), (u'1381', 143), (u'1554', 110), (u'4518', 18), (u'1728', 207), (u'900', 35), (u'207', 243), (u'1540', 70), (u'17289', 7), (u'1773', 127), (u'15687', 53), (u'13', 1060), (u'14727', 28), (u'2202', 16), (u'4024', 56), (u'1557', 489), (u'3641', 307), (u'3977', 32), (u'4356', 23), (u'11593', 20), (u'801', 23), (u'1706', 1), (u'1382', 33), (u'3029', 46), (u'4025', 47), (u'3536', 15), (u'17288', 13), (u'3675', 5), (u'1772', 320), (u'1286', 2

In [12]:
#mostra a quantidade de docentes doutores por IES

IESDocentePhdRDD = (DocenteHeadlessRDD
                   .filter(lambda rec: rec.CO_ESCOLARIDADE_DOCENTE == '5') 
                   .map(lambda rec: (rec.CO_IES,1))
                   .reduceByKey(lambda x,y: x+y)
                   )

print IESDocentePhdRDD.collect()

[(u'11', 208), (u'1788', 8), (u'1717', 12), (u'803', 5), (u'3962', 45), (u'15704', 16), (u'13467', 4), (u'11007', 1), (u'1771', 1), (u'4026', 20), (u'825', 3), (u'2200', 5), (u'3522', 6), (u'15696', 13), (u'68', 11), (u'1542', 54), (u'3975', 1), (u'2275', 12), (u'16543', 2), (u'976', 3), (u'3974', 78), (u'213', 67), (u'15697', 11), (u'10', 687), (u'12869', 7), (u'18023', 2), (u'14724', 127), (u'3039', 1), (u'1716', 11), (u'1789', 3), (u'3523', 21), (u'4030', 16), (u'802', 63), (u'824', 14), (u'4355', 1), (u'1381', 16), (u'1554', 15), (u'1728', 26), (u'900', 2), (u'207', 58), (u'1540', 11), (u'17289', 1), (u'1773', 18), (u'15687', 16), (u'13', 368), (u'14727', 5), (u'2202', 3), (u'4024', 22), (u'1557', 132), (u'3641', 109), (u'3977', 11), (u'4356', 1), (u'801', 1), (u'1706', 1), (u'1382', 15), (u'4518', 3), (u'4025', 21), (u'3536', 4), (u'17288', 2), (u'3675', 2), (u'1772', 60), (u'1286', 7), (u'15581', 25), (u'3998', 10), (u'1541', 14), (u'2687', 10), (u'826', 13), (u'206', 36), (u'967

In [14]:
#mostra a quantidade de docentes mestres por IES

IESDocenteMestradoRDD = (DocenteHeadlessRDD
                   .filter(lambda rec: rec.CO_ESCOLARIDADE_DOCENTE == '4') 
                   .map(lambda rec: (rec.CO_IES,1))
                   .reduceByKey(lambda x,y: x+y)
                   )

print IESDocenteMestradoRDD.collect()

[(u'11', 220), (u'1788', 44), (u'1542', 158), (u'803', 21), (u'3962', 98), (u'15704', 10), (u'13467', 5), (u'11007', 7), (u'1771', 5), (u'4026', 61), (u'825', 28), (u'2200', 28), (u'3522', 18), (u'15696', 28), (u'810', 36), (u'68', 22), (u'1717', 49), (u'3975', 1), (u'3643', 2), (u'2275', 24), (u'16543', 7), (u'976', 18), (u'3974', 238), (u'213', 68), (u'10', 734), (u'12869', 29), (u'18023', 7), (u'14724', 259), (u'15697', 16), (u'1716', 44), (u'1789', 27), (u'3523', 16), (u'4030', 61), (u'802', 89), (u'824', 42), (u'4355', 4), (u'1381', 102), (u'1554', 49), (u'4518', 15), (u'1728', 58), (u'900', 16), (u'207', 126), (u'1540', 33), (u'17289', 5), (u'1773', 66), (u'15687', 28), (u'13', 549), (u'14727', 17), (u'2202', 7), (u'4024', 31), (u'1557', 271), (u'3641', 154), (u'3977', 17), (u'11593', 5), (u'801', 1), (u'4356', 7), (u'1382', 16), (u'3029', 21), (u'826', 35), (u'4025', 22), (u'3536', 4), (u'17288', 7), (u'1772', 227), (u'1286', 10), (u'15581', 43), (u'3998', 90), (u'1541', 41), (u

In [16]:
#mostra a quantidade de docentes ATIVOS por IES

IESDocenteAtivoRDD = (DocenteHeadlessRDD
                   .filter(lambda rec: rec.CO_SITUACAO_DOCENTE == '1') 
                   .map(lambda rec: (rec.CO_IES,1))
                   .reduceByKey(lambda x,y: x+y)
                   )

print IESDocenteAtivoRDD.collect()

[(u'11', 464), (u'1788', 69), (u'1542', 259), (u'803', 43), (u'3962', 160), (u'15704', 30), (u'13467', 13), (u'11007', 20), (u'1771', 5), (u'4026', 100), (u'825', 44), (u'2200', 83), (u'3522', 27), (u'15696', 56), (u'810', 68), (u'68', 43), (u'1717', 108), (u'3975', 7), (u'3643', 2), (u'2275', 58), (u'16543', 9), (u'976', 32), (u'3974', 479), (u'213', 137), (u'15697', 28), (u'10', 1624), (u'12869', 26), (u'18023', 13), (u'14724', 442), (u'3039', 12), (u'1716', 94), (u'1789', 37), (u'3523', 38), (u'4030', 117), (u'802', 168), (u'824', 106), (u'4355', 10), (u'1381', 143), (u'1554', 110), (u'4518', 18), (u'1728', 207), (u'900', 35), (u'207', 220), (u'1540', 70), (u'17289', 7), (u'1773', 127), (u'15687', 53), (u'13', 1049), (u'14727', 28), (u'2202', 16), (u'4024', 56), (u'1557', 481), (u'3641', 303), (u'3977', 32), (u'4356', 21), (u'11593', 20), (u'801', 21), (u'1706', 1), (u'1382', 32), (u'3029', 43), (u'4025', 47), (u'3536', 15), (u'17288', 13), (u'3675', 5), (u'1772', 320), (u'1286', 20